In [1]:
# allow the notebook to access the parent directory so we can import the other modules
# https://stackoverflow.com/a/35273613
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# Data Preparation
-----

### Constants and Folder Paths

In [2]:
import os
dataset_folder_path = os.path.join("..", "files", "dataset")

### Load Data and Split into *Test*, *Train/Valid*

In [3]:
from data.DataSet import DataSet
dataset = DataSet()
dataset.load(dataset_folder_path, test_set_percentage=0, validation_set_percentage=0)

In [4]:
print(len(dataset.train_data))
print(len(dataset.test_data))

3600
0


### Data Preprocessing

In [5]:
from utils.preprocessing import *
from functools import partial
dataset.apply(apply_mean_centering)
dataset.apply(apply_unit_distance_normalization)
dataset.expand(reverse_digit_sequence)

In [6]:
print(len(dataset.train_data))
print(len(dataset.test_data))

7200
0


----------
## Test different sample rates and cross validate then compare the results to determine the optimal sampling rate to use
----------

In [15]:
NUM_SAMPLES_TO_TRY = [300, 200, 100, 75, 50, 25, 10]
N_FOLDS = 4
PARAM_NUM_EPOCHS = 20
PARAM_BATCH_SIZE = 300

In [16]:
import numpy as np
import pandas as pd
from utils.evaluation import cross_validate_model
from models.regularized_deep_gru import NaiveRegularizedDeepGRU

results = {}

for num_samples in NUM_SAMPLES_TO_TRY:
    print("\n\n\n-------------------------------")
    print("Evaluating Spline interpolation using %d samples" % num_samples)
    print("-------------------------------")
    # setup copy of data and evaluate its spline with the currently selected number of samples
    data = dataset.copy()
    data.apply(partial(spline_interpolate_and_resample, num_samples=num_samples))
    x = np.array(data.train_data)
    y = np.array(data.train_labels)
    # setup the model
    mymodel = NaiveRegularizedDeepGRU(x.shape[1:])
    mymodel.batch_size = PARAM_BATCH_SIZE
    mymodel.num_epochs = PARAM_NUM_EPOCHS
    # run cross validation
    scores = cross_validate_model(x, y, mymodel, N_FOLDS)
    results[num_samples] = scores
    




-------------------------------
Evaluating Spline interpolation using 300 samples
-------------------------------

....................
Cross validation fold [1]
....................

Train on 5400 samples, validate on 1800 samples
Epoch 1/20
5400/5400 [==============================] - 24s 4ms/step - loss: 2.0769 - categorical_accuracy: 0.2026 - val_loss: 2.0144 - val_categorical_accuracy: 0.2222
Epoch 2/20
5400/5400 [==============================] - 23s 4ms/step - loss: 1.9243 - categorical_accuracy: 0.2624 - val_loss: 1.7815 - val_categorical_accuracy: 0.3183
Epoch 3/20
5400/5400 [==============================] - 23s 4ms/step - loss: 1.6586 - categorical_accuracy: 0.3631 - val_loss: 1.6792 - val_categorical_accuracy: 0.3656
Epoch 4/20
5400/5400 [==============================] - 23s 4ms/step - loss: 1.3814 - categorical_accuracy: 0.4804 - val_loss: 1.2454 - val_categorical_accuracy: 0.4844
Epoch 5/20
5400/5400 [==============================] - 23s 4ms/step - loss: 1.1557 - cat

5400/5400 [==============================] - 23s 4ms/step - loss: 1.0210 - categorical_accuracy: 0.6331 - val_loss: 1.2794 - val_categorical_accuracy: 0.5828
Epoch 6/20
5400/5400 [==============================] - 23s 4ms/step - loss: 0.7550 - categorical_accuracy: 0.7522 - val_loss: 0.7820 - val_categorical_accuracy: 0.7617
Epoch 7/20
5400/5400 [==============================] - 23s 4ms/step - loss: 0.4040 - categorical_accuracy: 0.8750 - val_loss: 0.4882 - val_categorical_accuracy: 0.8372
Epoch 8/20
5400/5400 [==============================] - 23s 4ms/step - loss: 0.2032 - categorical_accuracy: 0.9389 - val_loss: 0.5013 - val_categorical_accuracy: 0.8539
Epoch 9/20
5400/5400 [==============================] - 23s 4ms/step - loss: 0.3197 - categorical_accuracy: 0.9072 - val_loss: 0.4468 - val_categorical_accuracy: 0.8706
Epoch 10/20
5400/5400 [==============================] - 23s 4ms/step - loss: 0.1443 - categorical_accuracy: 0.9598 - val_loss: 0.3768 - val_categorical_accuracy: 0.8

5400/5400 [==============================] - 14s 3ms/step - loss: 0.2878 - categorical_accuracy: 0.9278 - val_loss: 2.0828 - val_categorical_accuracy: 0.5039
Epoch 11/20
5400/5400 [==============================] - 14s 3ms/step - loss: 0.5079 - categorical_accuracy: 0.8576 - val_loss: 0.2238 - val_categorical_accuracy: 0.9439
Epoch 12/20
5400/5400 [==============================] - 14s 3ms/step - loss: 0.1216 - categorical_accuracy: 0.9663 - val_loss: 0.1639 - val_categorical_accuracy: 0.9639
Epoch 13/20
5400/5400 [==============================] - 14s 3ms/step - loss: 0.0753 - categorical_accuracy: 0.9791 - val_loss: 0.1329 - val_categorical_accuracy: 0.9728
Epoch 14/20
5400/5400 [==============================] - 14s 3ms/step - loss: 0.0564 - categorical_accuracy: 0.9863 - val_loss: 0.1481 - val_categorical_accuracy: 0.9628
Epoch 15/20
5400/5400 [==============================] - 14s 3ms/step - loss: 0.0543 - categorical_accuracy: 0.9854 - val_loss: 0.1270 - val_categorical_accuracy:

Epoch 16/20
5400/5400 [==============================] - 14s 3ms/step - loss: 0.0400 - categorical_accuracy: 0.9891 - val_loss: 0.1455 - val_categorical_accuracy: 0.9639
Epoch 17/20
5400/5400 [==============================] - 14s 3ms/step - loss: 0.0195 - categorical_accuracy: 0.9944 - val_loss: 0.1854 - val_categorical_accuracy: 0.9544
Epoch 18/20
5400/5400 [==============================] - 14s 3ms/step - loss: 0.0432 - categorical_accuracy: 0.9889 - val_loss: 0.2883 - val_categorical_accuracy: 0.9144
Epoch 19/20
5400/5400 [==============================] - 14s 3ms/step - loss: 0.1246 - categorical_accuracy: 0.9667 - val_loss: 0.1743 - val_categorical_accuracy: 0.9594
Epoch 20/20
1800/1800 [==============================] - 3s 2ms/step
categorical_accuracy: 96.83%

....................
Cross validation fold [4]
....................

Train on 5400 samples, validate on 1800 samples
Epoch 1/20
5400/5400 [==============================] - 16s 3ms/step - loss: 2.0780 - categorical_accura

1800/1800 [==============================] - 2s 950us/step
categorical_accuracy: 95.00%

....................
Cross validation fold [2]
....................

Train on 5400 samples, validate on 1800 samples
Epoch 1/20
5400/5400 [==============================] - 9s 2ms/step - loss: 2.0806 - categorical_accuracy: 0.2119 - val_loss: 1.9765 - val_categorical_accuracy: 0.2578
Epoch 2/20
5400/5400 [==============================] - 7s 1ms/step - loss: 1.8561 - categorical_accuracy: 0.3048 - val_loss: 1.5784 - val_categorical_accuracy: 0.3933
Epoch 3/20
5400/5400 [==============================] - 7s 1ms/step - loss: 1.4872 - categorical_accuracy: 0.4524 - val_loss: 1.1815 - val_categorical_accuracy: 0.5994
Epoch 4/20
5400/5400 [==============================] - 7s 1ms/step - loss: 1.1119 - categorical_accuracy: 0.6183 - val_loss: 0.7976 - val_categorical_accuracy: 0.7294
Epoch 5/20
5400/5400 [==============================] - 7s 1ms/step - loss: 0.6893 - categorical_accuracy: 0.7731 - val_lo

...

Train on 5400 samples, validate on 1800 samples
Epoch 1/20
5400/5400 [==============================] - 7s 1ms/step - loss: 2.0321 - categorical_accuracy: 0.2380 - val_loss: 1.7851 - val_categorical_accuracy: 0.3617
Epoch 2/20
5400/5400 [==============================] - 3s 646us/step - loss: 1.6390 - categorical_accuracy: 0.3989 - val_loss: 1.5165 - val_categorical_accuracy: 0.4556
Epoch 3/20
5400/5400 [==============================] - 3s 646us/step - loss: 1.2003 - categorical_accuracy: 0.5696 - val_loss: 0.8607 - val_categorical_accuracy: 0.7506
Epoch 4/20
5400/5400 [==============================] - 3s 646us/step - loss: 0.5589 - categorical_accuracy: 0.8135 - val_loss: 1.2118 - val_categorical_accuracy: 0.6583
Epoch 5/20
5400/5400 [==============================] - 3s 646us/step - loss: 0.9043 - categorical_accuracy: 0.7356 - val_loss: 0.5977 - val_categorical_accuracy: 0.8072
Epoch 6/20
5400/5400 [==============================] - 4s 649us/step - loss: 0.2326 - categorical_accur

5400/5400 [==============================] - 2s 338us/step - loss: 0.2382 - categorical_accuracy: 0.9252 - val_loss: 0.1441 - val_categorical_accuracy: 0.9622
Epoch 6/20
5400/5400 [==============================] - 2s 339us/step - loss: 0.3362 - categorical_accuracy: 0.8946 - val_loss: 0.1224 - val_categorical_accuracy: 0.9700
Epoch 7/20
5400/5400 [==============================] - 2s 338us/step - loss: 0.1335 - categorical_accuracy: 0.9624 - val_loss: 0.1759 - val_categorical_accuracy: 0.9528
Epoch 8/20
5400/5400 [==============================] - 2s 339us/step - loss: 0.0816 - categorical_accuracy: 0.9769 - val_loss: 0.1005 - val_categorical_accuracy: 0.9750
Epoch 9/20
5400/5400 [==============================] - 2s 339us/step - loss: 0.1724 - categorical_accuracy: 0.9502 - val_loss: 0.4860 - val_categorical_accuracy: 0.8711
Epoch 10/20
5400/5400 [==============================] - 2s 338us/step - loss: 0.2863 - categorical_accuracy: 0.9144 - val_loss: 0.0952 - val_categorical_accurac

Epoch 11/20
5400/5400 [==============================] - 2s 338us/step - loss: 0.6633 - categorical_accuracy: 0.7994 - val_loss: 0.3032 - val_categorical_accuracy: 0.9167
Epoch 12/20
5400/5400 [==============================] - 2s 340us/step - loss: 0.1283 - categorical_accuracy: 0.9657 - val_loss: 0.2049 - val_categorical_accuracy: 0.9450
Epoch 13/20
5400/5400 [==============================] - 2s 338us/step - loss: 0.0778 - categorical_accuracy: 0.9781 - val_loss: 0.1720 - val_categorical_accuracy: 0.9550
Epoch 14/20
5400/5400 [==============================] - 2s 338us/step - loss: 0.0733 - categorical_accuracy: 0.9807 - val_loss: 0.2074 - val_categorical_accuracy: 0.9483
Epoch 15/20
5400/5400 [==============================] - 2s 339us/step - loss: 0.0576 - categorical_accuracy: 0.9846 - val_loss: 0.2034 - val_categorical_accuracy: 0.9472
Epoch 16/20
5400/5400 [==============================] - 2s 339us/step - loss: 0.0337 - categorical_accuracy: 0.9917 - val_loss: 0.1813 - val_cat

Epoch 16/20
5400/5400 [==============================] - 1s 152us/step - loss: 0.0316 - categorical_accuracy: 0.9919 - val_loss: 0.1140 - val_categorical_accuracy: 0.9700
Epoch 17/20
5400/5400 [==============================] - 1s 153us/step - loss: 0.0261 - categorical_accuracy: 0.9928 - val_loss: 0.1051 - val_categorical_accuracy: 0.9767
Epoch 18/20
5400/5400 [==============================] - 1s 152us/step - loss: 0.0235 - categorical_accuracy: 0.9933 - val_loss: 0.1111 - val_categorical_accuracy: 0.9750
Epoch 19/20
5400/5400 [==============================] - 1s 151us/step - loss: 0.0194 - categorical_accuracy: 0.9948 - val_loss: 0.1119 - val_categorical_accuracy: 0.9744
Epoch 20/20
1800/1800 [==============================] - 0s 153us/step
categorical_accuracy: 70.89%

....................
Cross validation fold [2]
....................

Train on 5400 samples, validate on 1800 samples
Epoch 1/20
5400/5400 [==============================] - 5s 918us/step - loss: 1.9588 - categorical

Train on 5400 samples, validate on 1800 samples
Epoch 1/20
5400/5400 [==============================] - 5s 964us/step - loss: 1.9405 - categorical_accuracy: 0.3133 - val_loss: 2.1283 - val_categorical_accuracy: 0.2928
Epoch 2/20
5400/5400 [==============================] - 1s 152us/step - loss: 1.1499 - categorical_accuracy: 0.6083 - val_loss: 0.8771 - val_categorical_accuracy: 0.7233
Epoch 3/20
5400/5400 [==============================] - 1s 152us/step - loss: 0.6106 - categorical_accuracy: 0.8100 - val_loss: 0.6125 - val_categorical_accuracy: 0.8072
Epoch 4/20
5400/5400 [==============================] - 1s 152us/step - loss: 0.3857 - categorical_accuracy: 0.8828 - val_loss: 1.0547 - val_categorical_accuracy: 0.6728
Epoch 5/20
5400/5400 [==============================] - 1s 152us/step - loss: 0.2344 - categorical_accuracy: 0.9337 - val_loss: 0.5704 - val_categorical_accuracy: 0.8367
Epoch 6/20
5400/5400 [==============================] - 1s 152us/step - loss: 0.2584 - categorical_acc

In [17]:
results_df = pd.DataFrame([[key, np.mean(res), np.std(res)] for key,res in list(results.items())], columns=["Number of Samples", "Categorical Accuracy", "Std Deviation"])
results_df

,Number of Samples,Categorical Accuracy,Std Deviation
0,300,95.013889,2.092157
1,200,95.194444,1.357705
2,100,95.333333,0.368514
3,75,95.027778,2.395501
4,50,95.680556,1.860230
5,25,96.097222,1.643532
6,10,88.402778,10.279232
